In [1]:
from utils import *
import paddle
import paddle.nn.functional as F
import pandas as pd

train = load_dataset('all_data/train_full.txt')
texts = gather_text(train)
V = TextVectorizer()
V.build_vocab(texts)

Two vocabulary dictionaries have been built!
Please call X.vocab_to_idx | X.idx_to_vocab to find out more where [X] stands for the name you used for this TextVectorizer class.


In [2]:
def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model


def predict(model, data_loader):
    labels, predictions = [], []
    logits = model.predict(data_loader)
    for batch in data_loader:
        labels.extend(batch[-1].tolist())
        
    for batch in logits[0]:
        batch = paddle.to_tensor(batch)
        probs = F.softmax(batch, axis=1)
        preds = paddle.argmax(probs, axis=1).numpy().tolist()
        predictions.extend(preds)
        
    return predictions, labels


def get_accu_pre_recall_f1(preds, labels): 
    
    tp, fp, tn, fn, right = 0, 0, 0, 0, 0
    
    for (i, j) in zip(preds, labels):
        if i == 1 and j == 1:
            tp += 1
            right += 1
        elif i == 1 and j == 0:
            fp += 1
        elif i == 0 and j == 0:
            tn += 1
            right += 1
        else:
            fn += 1
    
    A = round(right/len(preds), 3)
    try:
        P = round(tp / (tp + fp), 3)
    except:
        return [A, 'Nan', 'Nan', 'Nan']
    R = round(tp / (tp + fn), 3)
    F1 = round(2 * P * R / (P + R), 3)
    
    return [A, P, R, F1]



In [3]:
from paddle_models.BoW import BoW
from paddle_models.CNN import CNN
from paddle_models.LSTM import LSTM
from paddle_models.GRU import GRU


def do_train_and_evaluate(train_path, 
                          network, 
                          epochs="3", 
                          dev_path='dev.tsv', 
                          test_path='test.tsv', 
                          batch_size=64,
                          save=False, 
                          save_dir='ckpt',
                          device="gpu", 
                          log_freq=500):
    
    paddle.set_device(device)
    
    if network.lower() == "bow":
        model = BoW(len(V.vocab_to_idx), 2)
        include_seq_len = False
    elif network.lower() == 'cnn':
        model = CNN(len(V.vocab_to_idx), 2)
        include_seq_len = False
    elif network.lower() == 'lstm':
        model = LSTM(len(V.vocab_to_idx), 2)
        include_seq_len = True
    elif network.lower() == 'gru':
        model = GRU(len(V.vocab_to_idx), 2)
        include_seq_len = True
    else:
        raise ValueError(f"Only supports: bow, cnn, lstm, gru, not {network}")

    train_set, dev_set, test_set = load_dataset([train_path, dev_path, test_path])
    trans_fn = get_trans_fn(V, include_seq_len)
    batchify_fn = get_batchify_fn(include_seq_len)
    train_loader = create_dataloader(train_set, trans_fn, batchify_fn)
    dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn)
    test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False)
    
    
    model = get_model(model)
    if save:
        model.fit(train_loader, dev_loader, epochs=epochs, batch_size=batch_size, 
                  verbose=2, log_freq=log_freq, save_dir=save_dir)
    else:
        model.fit(train_loader, dev_loader, epochs=epochs, batch_size=batch_size,
                  verbose=2, log_freq=log_freq)
        
    preds, labels = predict(model, test_loader)
    accu, prec, recall, f1 = get_accu_pre_recall_f1(preds, labels)
    
    return [accu, prec, recall, f1]

In [4]:
from IPython.display import clear_output

out = []

for size in ['0.5k', '1k', '2k', '4k', 'full']:
    for net in ['bow', 'cnn', 'lstm', 'gru']:
        print(f"Size: {size}; Model: {net}")
        res = do_train_and_evaluate(f'./all_data/train_{size}.txt', net, 3, device='gpu')
        out.append(['base', 'None', net, size] + res)
        clear_output(wait=True)

        columns = ['TrainType', 'EditType', 'ClfModel', 'TrainSize', 'Accuracy', 'Precision', 'Recall', 'F1']
        df = pd.DataFrame(out, columns=columns)
        df.to_excel('base_model_stats.xlsx', index=False)

Size: full; Model: gru
The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/3
step 109/109 - loss: 0.6052 - acc: 0.5447 - 22ms/step
Eval begin...
step 14/14 - loss: 0.5093 - acc: 0.7225 - 20ms/step
Eval samples: 872
Epoch 2/3
step 109/109 - loss: 0.2066 - acc: 0.8249 - 22ms/step
Eval begin...
step 14/14 - loss: 0.4217 - acc: 0.7844 - 19ms/step
Eval samples: 872
Epoch 3/3
step 109/109 - loss: 0.3137 - acc: 0.9389 - 22ms/step
Eval begin...
step 14/14 - loss: 0.4751 - acc: 0.7844 - 19ms/step
Eval samples: 872
Predict begin...
step 29/29 [==============================] - ETA: 1s - 50ms/ste - ETA: 0s - 34ms/ste - ETA: 0s - 29ms/ste - ETA: 0s - 27ms/ste - ETA: 0s - 25ms/ste - ETA: 0s - 24ms/ste - ETA: 0s - 23ms/ste - ETA: 0s - 23ms/ste - ETA: 0s - 23ms/ste - ETA: 0s - 22ms/ste - ETA: 0s - 22ms/ste - ETA: 0s - 21ms/ste - ETA: 0s - 20ms/ste - ETA: 0s - 19ms/ste - 19ms/step          
Predict samples: 1821


In [5]:
from IPython.display import clear_output

out = []

for aug_type in ['reda', 'reda_ngram']:
    for size in ['0.5k', '1k', '2k', '4k', 'full']:
        for net in ['bow', 'cnn', 'lstm', 'gru']:
            print(f"aug_type:{aug_type}. Size: {size}; Model: {net}")
            res = do_train_and_evaluate(f'./all_data/train_{size}_aug_{aug_type}.txt', net, 3, device="gpu")
            out.append([aug_type, 'All', net, size] + res)
            clear_output(wait=True)

            columns = ['TrainType', 'EditType', 'ClfModel', 'TrainSize', 'Accuracy', 'Precision', 'Recall', 'F1']
            df = pd.DataFrame(out, columns=columns)
            df.to_excel('aug_model_stats.xlsx', index=False)

aug_type:reda_ngram. Size: full; Model: gru
The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/3
step 500/637 - loss: 0.2716 - acc: 0.8289 - 23ms/step
step 637/637 - loss: 0.0980 - acc: 0.8564 - 23ms/step
Eval begin...
step 14/14 - loss: 0.5354 - acc: 0.7695 - 19ms/step
Eval samples: 872
Epoch 2/3
step 500/637 - loss: 0.0759 - acc: 0.9702 - 23ms/step
step 637/637 - loss: 0.0020 - acc: 0.9699 - 23ms/step
Eval begin...
step 14/14 - loss: 0.6863 - acc: 0.7626 - 19ms/step
Eval samples: 872
Epoch 3/3
step 500/637 - loss: 0.0887 - acc: 0.9813 - 23ms/step
step 637/637 - loss: 0.0925 - acc: 0.9816 - 23ms/step
Eval begin...
step 14/14 - loss: 1.5016 - acc: 0.7443 - 20ms/step
Eval samples: 872
Predict begin...
step 29/29 [==============================] - ETA: 1s - 51ms/ste - ETA: 0s - 35ms/ste - ETA: 0s - 30ms/ste - ETA: 0s - 27ms/ste - ETA: 0s - 25ms/ste - ETA: 0s - 24ms/ste - ETA: 0s - 24ms/ste - ETA: 0s - 23ms/ste - ETA: 0s -